In [24]:
%run constant.ipynb
import requests
import pandas as pd

# 获取历史股价数据的函数
def get_historical_data(symbol, access_token, period_type='year', period=5, frequency_type='daily', frequency=1):
    base_url = 'https://api.schwabapi.com/marketdata/v1/pricehistory'
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'symbol': symbol,
        'periodType': period_type,
        'period': period,
        'frequencyType': frequency_type,
        'frequency': frequency
    }

    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        # Parse JSON data into a DataFrame
        data = response.json()
        candles = data.get('candles', [])
        df = pd.DataFrame(candles)
        df['Date'] = pd.to_datetime(df['datetime'], unit='ms')  # Convert timestamps to datetime
        df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        return df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    else:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

# 计算 EMA 的函数
def calculate_ema(data, window):
    return data['Close'].ewm(span=window, adjust=False).mean()

# 计算并比较 EMA
def compare_emas(symbol, access_token):
    # 获取月线数据
    monthly_data = get_historical_data(symbol, access_token, period_type='year', period=10, frequency_type='monthly', frequency=1)
    monthly_ema = calculate_ema(monthly_data, 169).iloc[-1]

    # 获取周线数据
    weekly_data = get_historical_data(symbol, access_token, period_type='year', period=5, frequency_type='weekly', frequency=1)
    weekly_ema = calculate_ema(weekly_data, 169).iloc[-1]

    # 获取日线数据
    daily_data = get_historical_data(symbol, access_token, period_type='year', period=5, frequency_type='daily', frequency=1)
    daily_ema = calculate_ema(daily_data, 169).iloc[-1]

    # 获取最后一天的收盘价
    last_close = daily_data.iloc[-1]['Close']

    # 比较关系
    comparison = {
        'Last_Close': last_close,
        'Monthly_EMA_169': monthly_ema,
        'Weekly_EMA_169': weekly_ema,
        'Daily_EMA_169': daily_ema,
        'Comparison': {
            'Above_Monthly_EMA': last_close > monthly_ema,
            'Above_Weekly_EMA': last_close > weekly_ema,
            'Above_Daily_EMA': last_close > daily_ema
        }
    }

    return comparison

# 示例用法
if __name__ == "__main__":
    access_token = get_new_tokens()  # 替换为实际的获取 token 方法
    symbol = 'TMUS'  # 替换为需要分析的股票代码

    result = compare_emas(symbol, access_token)
    print("最后一天股价与EMA 169的关系:")
    print(result)


最后一天股价与EMA 169的关系:
{'Last_Close': np.float64(223.02), 'Monthly_EMA_169': np.float64(96.46227638519835), 'Weekly_EMA_169': np.float64(159.36167621329045), 'Daily_EMA_169': np.float64(204.67545691499362), 'Comparison': {'Above_Monthly_EMA': np.True_, 'Above_Weekly_EMA': np.True_, 'Above_Daily_EMA': np.True_}}


In [4]:
import sqlite3
import requests
import pandas as pd
%run constant.ipynb
# 获取历史股价数据的函数
def get_historical_data(symbol, access_token, period_type='year', period=5, frequency_type='daily', frequency=1):
    base_url = 'https://api.schwabapi.com/marketdata/v1/pricehistory'
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'symbol': symbol,
        'periodType': period_type,
        'period': period,
        'frequencyType': frequency_type,
        'frequency': frequency
    }

    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        candles = data.get('candles', [])
        df = pd.DataFrame(candles)
        df['Date'] = pd.to_datetime(df['datetime'], unit='ms')  # Convert timestamps to datetime
        df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        return df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    else:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

# 计算 EMA 的函数
def calculate_ema(data, window):
    return data['Close'].ewm(span=window, adjust=False).mean()

# 从数据库读取符号
def get_symbols_from_db(db_path="plan.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT symbol FROM EMA169")
    symbols = [row[0] for row in cursor.fetchall()]
    conn.close()
    return symbols

# 写入数据库
from datetime import datetime

# 写入数据库并更新日期
def update_ema_in_db(symbol, daily_ema, weekly_ema, monthly_ema, db_path='plan.db'):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Get the current date
    current_date = datetime.now().strftime('%Y-%m-%d')
    
    # Update EMA values and date
    cursor.execute("""
        UPDATE EMA169 
        SET daily = ?, weekly = ?, monthly = ?, date = ? 
        WHERE symbol = ?
    """, (daily_ema, weekly_ema, monthly_ema, current_date, symbol))
    
    conn.commit()
    conn.close()


# 计算并更新 EMA
def compare_and_update_emas(access_token, db_path=r"plan.db"):
    symbols = get_symbols_from_db(db_path)
    for symbol in symbols:
        try:
            # 获取月线数据
            monthly_data = get_historical_data(symbol, access_token, period_type='year', period=10, frequency_type='monthly', frequency=1)
            monthly_ema = calculate_ema(monthly_data, 169).iloc[-1]

            # 获取周线数据
            weekly_data = get_historical_data(symbol, access_token, period_type='year', period=5, frequency_type='weekly', frequency=1)
            weekly_ema = calculate_ema(weekly_data, 169).iloc[-1]

            # 获取日线数据
            daily_data = get_historical_data(symbol, access_token, period_type='year', period=5, frequency_type='daily', frequency=1)
            daily_ema = calculate_ema(daily_data, 169).iloc[-1]

            # 更新数据库
            update_ema_in_db(symbol, daily_ema, weekly_ema, monthly_ema, db_path)
            print(f"Updated EMA169 for {symbol}: Daily={daily_ema}, Weekly={weekly_ema}, Monthly={monthly_ema}")

        except Exception as e:
            print(f"Failed to update {symbol}: {e}")

# 示例用法
if __name__ == "__main__":
    access_token = get_new_tokens()  # 替换为实际的获取 token 方法
    compare_and_update_emas(access_token)


Updated EMA169 for MSFT: Daily=421.1232396594374, Weekly=346.11537475748725, Monthly=189.5478936721808
Updated EMA169 for AAPL: Daily=222.2388045835677, Weekly=180.07010599286033, Monthly=100.28393478441586
Updated EMA169 for PYPL: Daily=75.73706443577963, Weekly=91.31141488254308, Monthly=87.27011785471177
Updated EMA169 for CAT: Daily=361.8926807356638, Weekly=281.75114439990324, Monthly=172.8386532943073
Updated EMA169 for CVX: Daily=152.513502513519, Weekly=147.8345161426351, Monthly=122.59835108836123
Updated EMA169 for V: Daily=287.368111934914, Weekly=250.41253863755412, Monthly=163.521590566703
Updated EMA169 for META: Daily=537.1131645821552, Weekly=374.9306640280267, Monthly=217.38013919372966
Updated EMA169 for HON: Daily=213.3263892233114, Weekly=201.5979155778837, Monthly=158.72992160440023
Updated EMA169 for UNH: Daily=551.7431297506195, Weekly=497.0497439974406, Monthly=313.3157130849542
Updated EMA169 for GS: Daily=509.90101784628473, Weekly=398.3574987642615, Monthly=2